<a href="https://colab.research.google.com/github/jot240/Cassignment1/blob/master/DistrictHeating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install pulp

In [28]:
from pulp import *
import pandas as pd
import numpy as np
from math import *

In [10]:
   def read_excel_data(filename, sheet_name):
      data = pd.read_excel(filename, sheet_name=sheet_name, header=None)
      values = data.values
      return values

In [35]:
file1 = "InputDataEnergySmallInstance.xlsx"
v0 = read_excel_data(file1,"SourceNum")#location of first node
vi = read_excel_data(file1,"Nodes") #number of nodes
v_cord = read_excel_data(file1,"NodesCord") # nodes coordinates
theta_ij_fix = read_excel_data(file1,"vfix(thetaijfix)")#Vfix cost
theta_ij_var = read_excel_data(file1,"vvar(thetaijvar)")#Vvar cost
fixed_unit_cost = read_excel_data(file1,"FixedUnitCost")#cost for 1 unit
c_heat = read_excel_data(file1,"cheat(ciheat)")# heat cost for verticie i
c_ij_var = read_excel_data(file1,"cvar(cijvar)")#variable cost for edge ij
c_ij_om = read_excel_data(file1,"com(cijom)")# operations+maintenence cost for edge ij
c_ij_rev = read_excel_data(file1,"crev(cijrev)") #revenue for edge ij
t_full_load_hours = read_excel_data(file1,"Tflh(Tiflh)") #operation time of source verticis
beta = read_excel_data(file1,"Betta")#concurrence effect parameter
connection_quota = read_excel_data(file1,"Lambda") #connection quota parameter (lamda in paper)
alpha = read_excel_data(file1,"Alpha")# annuity factor for investment costs
d_ij_peak = read_excel_data(file1,"EdgesDemandPeak(dij)")#peak demand for edges ij (kW)
d_ij_annual = read_excel_data(file1,"EdgesDemandAnnual(Dij)")#annual demand for edges ij (kWh/year)
cap_ij_max = read_excel_data(file1,"Cmax(cijmax)")#maximum pipe capacity for edge (kW)
cap_source_max = read_excel_data(file1,"SourceMaxCap(Qimax)")#(Qimax)Maximum heat generation capacity for source i (kW)
p_umd = read_excel_data(file1,"pumd(pijumd)")#(pumd)unmet demand penalty for edge ij


In [31]:
 #Create decision Variables
node_num = vi[0][0]
source_loc = v0[0][0]
set_I = [*range(node_num)] #nodes
distances_ij = np.zeros(shape=(node_num,node_num))
print(v_cord[0][1])
for i in set_I:
  for j in set_I:
    distances_ij[i][j] = sqrt((v_cord[i][0]- v_cord[j][0])**2 + (v_cord[i][1]-v_cord[j][1])**2)

x_ij =  LpVariable.dicts('x', (set_I, set_I ),0,1, "Binary")
p_in_ij = LpVariable.dicts('pin', (set_I,set_I),lowBound=0)
p_out_ij = LpVariable.dicts('pin', (set_I,set_I),lowBound=0)



30


In [36]:
print(x_ij)
#make problem
district_heating = LpProblem("district_heating",LpMinimize)
revenue = lpSum(x_ij[i][j]* d_ij_annual[i-1][j-1]*c_ij_rev[i-1][j-1]*connection_quota for i in set_I for j in set_I)
heat_gen_cost = lpSum(t_full_load_hours[0][0] * c_heat[source_loc][0] * p_in_ij[source_loc][j]/ beta[0][0] for j in set_I)
maintenance_cost = lpSum(c_ij_om[i-1][j-1]* distances_ij[i][j]*x_ij[i][j] for i in set_I for j in set_I)
fixed_invest_cost = lpSum(fixed_unit_cost[0][0] * distances_ij[i][j]* alpha[0][0] *x_ij[i][j] for i in set_I for j in set_I )
variable_invest_cost = lpSum(c_ij_var[i-1][j-1]* distances_ij[i][j]* alpha[0][0]* p_in_ij[i][j] for i in set_I for j in set_I)
umd_pen_cost = lpSum(d_ij_annual[i-1][j-1] * p_umd[i-1][j-1]* (1-x_ij[i][j]-x_ij[j][i]) for i in set_I for j in set_I )


{0: {0: x_0_0, 1: x_0_1, 2: x_0_2, 3: x_0_3, 4: x_0_4, 5: x_0_5, 6: x_0_6, 7: x_0_7}, 1: {0: x_1_0, 1: x_1_1, 2: x_1_2, 3: x_1_3, 4: x_1_4, 5: x_1_5, 6: x_1_6, 7: x_1_7}, 2: {0: x_2_0, 1: x_2_1, 2: x_2_2, 3: x_2_3, 4: x_2_4, 5: x_2_5, 6: x_2_6, 7: x_2_7}, 3: {0: x_3_0, 1: x_3_1, 2: x_3_2, 3: x_3_3, 4: x_3_4, 5: x_3_5, 6: x_3_6, 7: x_3_7}, 4: {0: x_4_0, 1: x_4_1, 2: x_4_2, 3: x_4_3, 4: x_4_4, 5: x_4_5, 6: x_4_6, 7: x_4_7}, 5: {0: x_5_0, 1: x_5_1, 2: x_5_2, 3: x_5_3, 4: x_5_4, 5: x_5_5, 6: x_5_6, 7: x_5_7}, 6: {0: x_6_0, 1: x_6_1, 2: x_6_2, 3: x_6_3, 4: x_6_4, 5: x_6_5, 6: x_6_6, 7: x_6_7}, 7: {0: x_7_0, 1: x_7_1, 2: x_7_2, 3: x_7_3, 4: x_7_4, 5: x_7_5, 6: x_7_6, 7: x_7_7}}


In [ ]:
district_heating += heat_gen_cost + maintenance_cost + fixed_invest_cost + variable_invest_cost + umd_pen_cost - revenue

In [41]:
#constraints
#Tree structure
district_heating += lpSum(x_ij[i][j] for i in set_I for j in set_I) == node_num -1
#Unidirectionality
district_heating += lpSum(x_ij[i][j] + x_ij[j][i]) <= 1
#Demand Satisfaction
for i in set_I:
  for j in set_I:
    district_heating += lpSum((1 - distances_ij[i][j]*theta_ij_var[i-1][j-1])*p_in_ij[i][j] - p_out_ij[i][j] ) == lpSum((distances_ij[i][j] *theta_ij_fix[i-1][j-1] + d_ij_peak[i-1][j-1]*connection_quota *beta)*x_ij[i][j] )
#flow equilibrium
for i in set_I:
  for j in set_I:
    if i != source_loc:
      district_heating += lpSum(p_in_ij[i][j]-p_out_ij[i][j]) == 0
#edge capacity
for i in set_I:
  for j in set_I:
    district_heating += lpSum(p_in_ij[i][j]) <= lpSum(x_ij[i][j]*cap_ij_max[i-1][j-1])
#source structure
for j in set_I:
  district_heating += lpSum(x_ij[j][source_loc]) ==0
#Source heat generation capacity
for j in set_I:
  district_heating += lpSum(p_in_ij[source_loc][j] - p_out_ij[source_loc][j] ) <= cap_source_max[j][0]
#Tour elimination
for i in set_I: 
  for j in set_I:
    if i != source_loc:
      district_heating +=lpSum(x_ij[j][i]) >= 1


